In [2]:
import os
import pandas as pd
import sys
import time
from pathlib import Path
import snscrape.modules.twitter as sntwitter

# handle with care!
import warnings
warnings.filterwarnings('ignore')

## Used this for twitter API, no need for it now!
# consumer_key = 'lcGh2Z7t1BQEOdpqn2S1Jce8O'
# consumer_secret = 'IyJh2frDC6Xb7M1FV68v4qLHtOTRv23XKHWoRZuchpC1dtSwop'
# access_token = '15233837-i5Rh8eWV5OhpEWeSBIyhoxNxEMaCrMUOSsg932F5B'
# access_token_secret = 'Di6cPb6CIpwZX0kQ4GjVm5j7j6lssarlwGwTca8l3a0bs'


# auth = tw.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_token_secret)
# api = tw.API(auth, wait_on_rate_limit=True)


dubai_gov = ['@DXBMediaOffice',
             '@SmartDubai', '@dghr_gov', '@DMunicipality', '@LAD_Dubai', '@CDA_Dubai', '@DHA_Dubai',
             '@KHDA', '@Dubai_Ambulance', '@dubaimediainc', '@DubaiWomenEst', '@NedaaDXB',
             '@DubaiPoliceHQ', '@GDRFADUBAI', '@DubaiStatistics']

kuwait_gov = ['@SocialDOffice', '@KuwaitiCM', '@CitraKuwait', '@GPFKW', '@CAIT_KW', '@CSBkw', '@Csc_Kw', '@Moi_kuw',
              '@KUWAIT_MOH', '@mosal_q8', '@MOInformation', '@mockuwait', '@mpw_kw', '@youthkuwait', '@MajlesAlOmmah',
              '@kuwmun', '@OfficialKfsd', '@manpower_kwt', '@kuna_ar', '@NewKuwaitGov']

# check handles for twitter
kuwait_influencers = ['@Alafasy', '@NabilAlawadhy', '@DrAlnefisi', '@TfTeeeSH', '@mbuyabis', '@safaalhashem', '@MarzouqAlghanim',
                      '@FaisalAlBasri', '@bourashed', '@nohastyleicon', '@Bibii6363', '@Dr__ALKANDARI', '@ThamerAlsuwait', '@WahabAlbabtain',
                      '@ladyrainq8', '@Tabtabaee', '@Ran01da', '@MHALHAJARAF', '@alhuwaila', '@BO3LOSH81', '@Abdllah_fhhad', '@watan_alnhhar',
                      '@ferasaloan', '@OdahAlodah', '@bntblad29160144', '@Mishari77', '@fatma_aljenaei', '@kuwait_PLg', '@saeedtawfiqi',
                      '@NasrAlDousari', '@kuwait00660830', '@kuniv_j', '@AlroumiDr', '@Thephoenix_24'
                      ]
                      
dubai_influencers = ["@mustafa_agha",
                     "@Shamma_Hamdan",
                     "@Lara",
                     "@Lojain_omran",
                     "@mobeid",
                     "@dbelhoul",
                     "@WazhmaAyoubi",
                     "@LSLofficial",
                     "@DubaiNameShame",
                     "@TwoToneDXB",
                     "@krisfade",
                     "@Mahra",
                     "@h_alrahoomi",
                     "@DJBLISS",
                     "@_SocialCompany",
                     "@M_BinTheneya",
                     "@afmfilms",
                     "@HishamWyne",
                     "@zizzleblog",
                     "@literallyanika",
                     "@1magicphil",
                     "@Sara_Falaknaz",
                     "@GrosvenorHouse",
                     "@OsamaAlshafar",
                     "@FryingPanTours",
                     "@EatDrinkStayDXB",
                     "@RiggsGolf",
                     "@GLORYGIRLFIT",
                     "@samiraOlfat",
                     "@farhanabodi",
                     "@HassanSoukar",
                     "@amrutant",
                     "@Cookwithzahra",
                     "@ayeina_official",
                     "@BloomerBrittany",
                     ]

since_date = '2020-01-03'
until_date = '2022-06-03'
count = 5000


## Tweet scraper for single user and single keyword

In [3]:
def user_query_tweets(username, text_query, count, since_date, until_date, lang):
    tweetlist2 = []
    # print('from:'+username+' '+text_query+' since:'+since_date+' until:'+until_date)
    for i, tweet in enumerate(sntwitter.TwitterSearchScraper('from:'+username+' '+text_query+' since:'+since_date+' until:'+until_date +' lang:'+lang).get_items()):
        if i > count:
            break
        if tweet.content:
            tweetlist2.append([tweet.date,
                               tweet.id,
                               tweet.rawContent,
                               tweet.user.username,
                               tweet.inReplyToUser,
                            #    tweet.inReplyToTweetId,
                               tweet.replyCount,
                               tweet.retweetCount,
                               tweet.likeCount,
                               tweet.user.followersCount,
                               tweet.user.friendsCount,
                               tweet.user.statusesCount,
                               tweet.lang])
    # Creating a dataframe from the tweets list above
    tweets_df2 = pd.DataFrame(
        tweetlist2, columns=['Datetime',
                             'Tweet Id',
                             'Text',
                             'Username',
                             'in_reply_to_user_id',
                            #  'in_reply_to_status_id',
                             'reply_count',
                             'retweet_count',
                             'favorite_count',
                             'follower_count',
                             'friends_count',
                             'total_tweets',
                             'lang'])
    return tweets_df2


## Check twitter handles

In [4]:
check_users = []
lang = 'ar'
for user in dubai_influencers:
    df = user_query_tweets(user, '',1, since_date, until_date,lang)
    check_users.append(df.shape[0] > 0)

res = dict(zip(dubai_influencers, check_users))
#print false ones
print({k: v for (k,v) in res.items() if v is False})

{'@Lara': False, '@LSLofficial': False, '@DubaiNameShame': False, '@TwoToneDXB': False, '@krisfade': False, '@DJBLISS': False, '@_SocialCompany': False, '@HishamWyne': False, '@zizzleblog': False, '@literallyanika': False, '@1magicphil': False, '@GrosvenorHouse': False, '@FryingPanTours': False, '@EatDrinkStayDXB': False, '@RiggsGolf': False, '@GLORYGIRLFIT': False, '@samiraOlfat': False, '@farhanabodi': False, '@amrutant': False, '@Cookwithzahra': False, '@ayeina_official': False, '@BloomerBrittany': False}


## Main Twitter Scraper

In [5]:
tweet_info = ['Datetime',
                             'Tweet Id',
                             'Text',
                             'Username',
                             'in_reply_to_user_id',
                            #  'in_reply_to_status_id',
                             'reply_count',
                             'retweet_count',
                             'favorite_count',
                             'follower_count',
                             'friends_count',
                             'total_tweets',
                             'lang']


def user_query_tweets_to_csv(file_path,username_list,query_terms,count,since_date,until_date, lang):
    tweets_df=pd.DataFrame(list(), columns=tweet_info)
    tweet_cnt = []
    for i,query in enumerate(query_terms):
        for ii,name in enumerate(username_list):
            data=user_query_tweets(name ,query,count,since_date,until_date, lang)
            if not data.empty:
                tweets_df=pd.concat([tweets_df, data], axis = 0)
                tweet_cnt.append([query, name, data.shape[0]])

    tweets_df.to_csv(file_path,index=False,encoding='utf-8')
    return tweets_df, tweet_cnt

In [6]:
# query_terms = ['کورونا','كوفيد','Covid-19','العمال غير المواطنين','Covid','مواطن','المغتربين',
#                'منظمة غير حكومية','تمكين المواطن','التنمية المجتمعية المستدامة','الدمقرطة',
#               'منظمة الناس','مشاركة المواطن','المشاركة المدنية','سياسة المجتمع','بناء القدرات',
#               'منظمة شعبية','الرأسمالية الاجتماعية','التنمية التشاركية','حقوق التصويت','التعددية','مجتمع مدني مستقل'
#                ,'المناصرة','العدالة البديلة','التأثير العام','الشفافية السياسية','الشفافية الاقتصادية','التربية المدنية']

query_terms_ar = ['کورونا','كوفيد',
    'المغتربين', 'العمالة الوافدة','عامل وافد', 'العمال المهاجرين', 'العمال الوطنيين', 'العمال غير المواطنين' ,
 'مواطن', 'غير مواطن'  , 'مقيم' , 'ساكن', 'التطعيم ضد كورونا', 'لقاح كورونا', 'الحجر صحي', 'دخول المستشفى' ]

query_terms_en = ['Coronavirus', 'Covid-19', 'Vaccine', 'Immunization', 'expat worker', 'expats',
 'expat workers', 'migrant workers', 'national workers', 'non-national workers',
  'citizen', 'non-citizen', 'resident', 'dweller', 'Quarantine', 'hospitalize']


In [7]:
mydir = './data_jan26/'
if not os.path.exists(mydir):
    os.mkdir(mydir)

In [ ]:
filename='test.csv'
file_path = mydir + filename
# df, tweet_cnt = user_query_tweets_to_csv(file_path, dubai_influencers, query_terms_ar[:2], count, since_date, until_date)
test, tweet_cnt = user_query_tweets_to_csv(file_path, dubai_influencers, [''], count, since_date, until_date, 'ar')

In [ ]:
test.shape

(18468, 12)

In [ ]:
test.shape

(946, 12)

# Automated extract all tweets, no keywords

In [8]:
## Automated tool
def generate_all_nokey(total_iter: dict, total_queries: dict, mydir, spaces = 1):

    tz = time.perf_counter()
    for iter in total_iter.keys():
        with pd.ExcelWriter(mydir + f"{iter}_report.xlsx", engine='xlsxwriter') as writer:
            for lang in total_queries.keys():
                users = total_iter.get(iter)
                keywords = ['']
                filename = f"{iter}_{lang}.csv"
                file_path = mydir + filename

                df, tweet_cnt = user_query_tweets_to_csv(file_path, users, keywords, count, since_date, until_date, lang)
                print(f"Extracted tweets for following category {iter}_{lang}")
                print(f'Total Number of tweets : {df.shape[0]}')

                # generating report -- more dfs could be added here
                xx = pd.DataFrame(tweet_cnt, columns=[
                                'keyword', 'handle', 'tweet count'])
                df_handle = pd.DataFrame(users, columns=['User']).merge(xx.drop(columns='keyword').groupby(
                    'handle').sum(), how='left', left_on='User', right_on='handle').fillna(0.0)
                df_keywords = pd.DataFrame(keywords, columns=['keyword']).merge(xx.drop(
                    columns='handle').groupby('keyword').sum(), how='left', on='keyword').fillna(0.0)

                dfs = [df_handle, df_keywords]
                col = 0
                for dataframe in dfs:
                    dataframe.to_excel(writer,sheet_name= f'counts_{lang}',startrow=1 , startcol=col)   
                    # row = row + len(dataframe.index) + spaces + 1
                    col = col + dataframe.shape[1] + spaces + 1


    t = time.perf_counter() - tz
    print(f"Total Elapsed time to generate all iterations: {t:.2f}s")


In [12]:
OUTPUT = './data_march2nd/'
if not os.path.exists(OUTPUT):
    os.mkdir(OUTPUT)

since_date = '2020-01-03'
until_date = '2022-06-03'
# total tweets extracted for each user
count = 10000


total_iter = {
    # "kuwait_inf": kuwait_influencers,
    # "kuwait_gov": kuwait_gov,
    "dubai_inf": dubai_influencers,
    # "dubai_gov": dubai_gov
}
total_queries = {'ar': query_terms_ar, 'en': query_terms_en}

generate_all_nokey(total_iter, total_queries, OUTPUT)

Extracted tweets for following category dubai_inf_ar
Total Number of tweets : 18462
Extracted tweets for following category dubai_inf_en
Total Number of tweets : 14821
Extracted tweets for following category dubai_gov_ar
Total Number of tweets : 26354
Extracted tweets for following category dubai_gov_en
Total Number of tweets : 19086
Total Elapsed time to generate all iterations: 2229.37s


# Automated using keywords

Report structure:

Let's say for Kuwait influencers --- 4 total excel files at the end

Consisted of multiple sheets:
* count_en: query terms, usernames, total tweets extracted AND tweet_count
* count_ar: query terms, usernames, total tweets extracted AND tweet_count
* More info on users and number of likes?! *to ask stakeholders

In [93]:
xx = pd.DataFrame(tweet_cnt, columns=['keyword', 'handle', 'tweet count'])
print(f'Total Number of tweets in {filename}: {df.shape[0]}')

Total Number of tweets in kuwait_inf_ar.csv: 7527


In [94]:
xx

,keyword,handle,tweet count
0,کورونا,@Alafasy,20
1,کورونا,@NabilAlawadhy,36
2,کورونا,@DrAlnefisi,1
3,کورونا,@TfTeeeSH,3371
4,کورونا,@mbuyabis,188
...,...,...,...
133,لقاح كورونا,@Mishari77,1
134,لقاح كورونا,@saeedtawfiqi,6
135,الحجر صحي,@TfTeeeSH,1
136,دخول المستشفى,@TfTeeeSH,6


## Automated function to get tweets by group and generate report

In [11]:
## Automated tool
def generate_all(total_iter: dict, total_queries: dict, mydir, spaces = 1):

    if not os.path.exists(mydir):
        os.mkdir(mydir)

    tz = time.perf_counter()
    for iter in total_iter.keys():
        with pd.ExcelWriter(mydir + f"{iter}_report.xlsx", engine='xlsxwriter') as writer:
            for lang in total_queries.keys():
                users = total_iter.get(iter)
                keywords = total_queries.get(lang)
                filename = f"{iter}_{lang}.csv"
                file_path = mydir + filename

                df, tweet_cnt = user_query_tweets_to_csv(file_path, users, keywords, count, since_date, until_date, lang)
                print(f"Extracted tweets for following category {iter}_{lang}")
                print(f'Total Number of tweets : {df.shape[0]}')

                # generating report -- more dfs could be added here
                xx = pd.DataFrame(tweet_cnt, columns=[
                                'keyword', 'handle', 'tweet count'])
                df_handle = pd.DataFrame(users, columns=['User']).merge(xx.drop(columns='keyword').groupby(
                    'handle').sum(), how='left', left_on='User', right_on='handle').fillna(0.0)
                df_keywords = pd.DataFrame(keywords, columns=['keyword']).merge(xx.drop(
                    columns='handle').groupby('keyword').sum(), how='left', on='keyword').fillna(0.0)

                dfs = [df_handle, df_keywords]
                col = 0
                for dataframe in dfs:
                    dataframe.to_excel(writer,sheet_name= f'counts_{lang}',startrow=1 , startcol=col)   
                    # row = row + len(dataframe.index) + spaces + 1
                    col = col + dataframe.shape[1] + spaces + 1


    t = time.perf_counter() - tz
    print(f"Total Elapsed time to generate all iterations: {t:.2f}s")


In [13]:
# first test
total_iter = {
    # "kuwait_inf": kuwait_influencers,
    # "kuwait_gov": kuwait_gov,
    "dubai_inf": dubai_influencers,
    # "dubai_gov": dubai_gov
}
# total_queries = {'ar': query_terms_ar, 'eng': query_terms_en}
total_queries = {'eng': query_terms_en}

mydir = './data_latest/'

generate_all(total_iter, total_queries, mydir)

Extracted tweets for following category dubai_inf_eng
Total Number of tweets : 0
Total Elapsed time to generate all iterations: 488.74s
